In [4]:
#Installs the timm library for pre-trained PyTorch models.
!pip install timm


In [5]:
from timm.models.layers import to_2tuple, trunc_normal_, DropPath
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class PatchEmbed(nn.Module):
    def __init__(self, img_size=224, patch_size=16, in_chan=3, embed_dim=768, multi_conv=False, use_simple_conv=False):
        super().__init__()
        img_size = to_2tuple(img_size)
        patch_size = to_2tuple(patch_size)
        num_patches = (img_size[1] // patch_size[1]) * (img_size[0] // patch_size[0])
        self.img_size = img_size
        self.patch_size = patch_size
        self.num_patches = num_patches

        if multi_conv and not use_simple_conv:
            if patch_size[0] == 24:
                self.proj = nn.Sequential(
                    nn.Conv2d(in_chan, embed_dim // 4, kernel_size=7, stride=3, padding=2),
                    nn.ReLU(inplace=True),
                    nn.Conv2d(embed_dim // 4, embed_dim // 2, kernel_size=3, stride=2, padding=1),
                    nn.ReLU(inplace=True),
                    nn.Conv2d(embed_dim // 2, embed_dim, kernel_size=3, stride=1, padding=1),
                )
            elif patch_size[0] == 12:
                self.proj = nn.Sequential(
                    nn.Conv2d(in_chan, embed_dim // 4, kernel_size=7, stride=4, padding=3),
                    nn.ReLU(inplace=True),
                    nn.Conv2d(embed_dim // 4, embed_dim // 2, kernel_size=3, stride=2, padding=1),
                    nn.ReLU(inplace=True),
                    nn.Conv2d(embed_dim // 2, embed_dim, kernel_size=3, stride=1, padding=1),
                )
            elif patch_size[0] == 4:
                self.proj = nn.Sequential(
                    nn.Conv2d(in_chan, embed_dim // 4, kernel_size=7, stride=2, padding=3),
                    nn.ReLU(inplace=True),
                    nn.Conv2d(embed_dim // 4, embed_dim // 2, kernel_size=3, stride=2, padding=1),
                    nn.ReLU(inplace=True),
                    nn.Conv2d(embed_dim // 2, embed_dim, kernel_size=3, stride=1, padding=1),
                )
            else:
                raise ValueError(f"Unsupported patch size {patch_size[0]}")
        elif use_simple_conv:
            # Simple convolutional embedding
            self.proj = nn.Conv2d(in_chan, embed_dim, kernel_size=patch_size, stride=patch_size)
        else:
            # Default behavior if multi_conv is not used
            self.proj = nn.Conv2d(in_chan, embed_dim, kernel_size=patch_size, stride=patch_size)

    def forward(self, x, extra_padding=False):
        B, C, H, W = x.shape
        print(f"Input shape: {x.shape}")
        if extra_padding and (H % self.patch_size[0] != 0 or W % self.patch_size[1] != 0):
            p_l = (self.patch_size[1] - W % self.patch_size[1]) // 2
            p_r = (self.patch_size[1] - W % self.patch_size[1]) - p_l
            p_t = (self.patch_size[0] - H % self.patch_size[0]) // 2
            p_b = (self.patch_size[0] - H % self.patch_size[0]) - p_t
            x = F.pad(x, (p_l, p_r, p_t, p_b))
            print(f"Padded shape: {x.shape}")

        x = self.proj(x)  # Apply the projection (whether multi-conv or simple-conv)
        x = x.flatten(2).transpose(1, 2)  # Flatten and transpose for transformer input

        return x

# Utility function to convert values to tuple if they aren't already
def to_2tuple(x):
    if isinstance(x, tuple):
        return x
    return (x, x)


In [7]:
import copy
import torch
import torch.nn as nn
import torch.nn.functional as F

# Utility function to create N identical layers
def clones(module, N):
    return nn.ModuleList([copy.deepcopy(module) for _ in range(N)])

class LayerNorm(nn.Module):
    """Construct a layernorm module."""
    def __init__(self, features, eps=1e-6):
        super(LayerNorm, self).__init__()
        self.a_2 = nn.Parameter(torch.ones(features))
        self.b_2 = nn.Parameter(torch.zeros(features))
        self.eps = eps

    def forward(self, x):
        mean = x.mean(-1, keepdim=True)
        std = x.std(-1, keepdim=True)
        return self.a_2 * (x - mean) / (std + self.eps) + self.b_2

class SublayerConnection(nn.Module):
    """A residual connection followed by a layer norm."""
    def __init__(self, size, dropout):
        super(SublayerConnection, self).__init__()
        self.norm = LayerNorm(size)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, sublayer):
        return x + self.dropout(sublayer(self.norm(x)))

class EncoderLayer(nn.Module):
    """Encoder is made up of self-attn and feed forward (defined below)"""
    def __init__(self, size, self_attn, feed_forward, dropout):
        super(EncoderLayer, self).__init__()
        self.self_attn = self_attn
        self.feed_forward = feed_forward
        self.sublayer = clones(SublayerConnection(size, dropout), 2)
        self.size = size

    def forward(self, x, mask):
        x = self.sublayer[0](x, lambda x: self.self_attn(x, x, x, mask))
        return self.sublayer[1](x, self.feed_forward)

class Encoder(nn.Module):
    """Core encoder is a stack of N layers"""
    def __init__(self, layer, N):
        super(Encoder, self).__init__()
        self.layers = clones(layer, N)
        self.norm = LayerNorm(layer.size)
        
    def forward(self, x, mask):
        for layer in self.layers:
            x = layer(x, mask)
        return self.norm(x)

class Sparsifier(nn.Module):
    """Applies sparsification to the encoder output."""
    def __init__(self, threshold=0.1):
        super(Sparsifier, self).__init__()
        self.threshold = threshold

    def forward(self, x):
        mask = (torch.abs(x) > self.threshold).float()
        return x * mask

# Assuming you already have the PatchEmbed class from your previous code

class VisionTransformer(nn.Module):
    def __init__(self, img_size=(224, 224), patch_size=(16, 24), in_chans=3, num_classes=1000,
                 embed_dim=[384, 768], depth=([1, 4, 0], [1, 4, 0], [1, 4, 0]), num_heads=[12, 12],
                 mlp_ratio=[4, 4, 1], qkv_bias=False, qk_scale=None, drop_rate=0.,
                 attn_drop_rate=0., drop_path_rate=0., norm_layer=nn.LayerNorm, sparsify=False, multi_conv=False):
        super(VisionTransformer, self).__init__()

        self.num_classes = num_classes
        self.img_size = img_size if isinstance(img_size, list) else to_2tuple(img_size)
        self.num_branches = len(patch_size)

        self.patch_embed = nn.ModuleList([PatchEmbed(img_size=im_s, patch_size=p, in_chan=in_chans,
                                                     embed_dim=d, multi_conv=multi_conv)
                                          for im_s, p, d in zip(img_size, patch_size, embed_dim)])

        self.pos_embed = nn.ParameterList([nn.Parameter(torch.zeros(1, 1 + _compute_num_patches(im_s, p), d))
                                           for im_s, p, d in zip(img_size, patch_size, embed_dim)])

        self.cls_token = nn.ParameterList([nn.Parameter(torch.zeros(1, 1, d)) for d in embed_dim])
        self.pos_drop = nn.Dropout(p=drop_rate)

        # Create encoder layers
        self.encoders = nn.ModuleList([Encoder(EncoderLayer(embed_dim[i], MultiHeadedAttention(num_heads[i], embed_dim[i]),
                                                     nn.Sequential(nn.Linear(embed_dim[i], embed_dim[i] * mlp_ratio[i]),
                                                                   nn.ReLU(),
                                                                   nn.Linear(embed_dim[i] * mlp_ratio[i], embed_dim[i])),
                                                     drop_rate), depth[i][-1])
                                       for i in range(self.num_branches)])
        
        # Sparsifier module for each branch
        self.sparsifiers = nn.ModuleList([Sparsifier() if sparsify else nn.Identity() for _ in range(self.num_branches)])

        self.norm = nn.ModuleList([norm_layer(d) for d in embed_dim])
        self.head = nn.ModuleList([nn.Linear(d, num_classes) if num_classes > 0 else nn.Identity() for d in embed_dim])

        for i in range(self.num_branches):
            trunc_normal_(self.pos_embed[i], std=.02)
            trunc_normal_(self.cls_token[i], std=.02)

        self.apply(self._init_weights)

    def _init_weights(self, m):
        if isinstance(m, nn.Linear):
            trunc_normal_(m.weight, std=.02)
            nn.init.constant_(m.bias, 0)
        elif isinstance(m, nn.LayerNorm):
            nn.init.constant_(m.bias, 0)
            nn.init.constant_(m.weight, 1.0)

    def forward_features(self, x):
        B, C, H, W = x.shape
        xs = []
        for i in range(self.num_branches):
            x_ = torch.nn.functional.interpolate(x, size=(self.img_size[i], self.img_size[i]), mode='bicubic') if H != self.img_size[i] else x
            tmp = self.patch_embed[i](x_)
            cls_tokens = self.cls_token[i].expand(B, -1, -1)
            tmp = torch.cat((cls_tokens, tmp), dim=1)
            tmp = tmp + self.pos_embed[i]
            tmp = self.pos_drop(tmp)
            tmp = self.encoders[i](tmp, None)  # Encoder processing
            tmp = self.sparsifiers[i](tmp)  # Apply sparsification
            xs.append(tmp)

        xs = [self.norm[i](x) for i, x in enumerate(xs)]
        out = [x[:, 0] for x in xs]

        return out

    def forward(self, x):
        xs = self.forward_features(x)
        ce_logits = [self.head[i](x) for i, x in enumerate(xs)]
        ce_logits = torch.mean(torch.stack(ce_logits, dim=0), dim=0)
        return ce_logits


In [8]:
class CrossAttention(nn.Module):
    def __init__(self, dim, num_heads=8, qkv_bias=False, qk_scale=None, attn_drop=0., proj_drop=0., patch_size=16):
        super().__init__()
        self.num_heads = num_heads
        head_dim = dim // num_heads
        self.scale = qk_scale or head_dim ** -0.5

        self.wq = nn.Linear(dim, dim, bias=qkv_bias)
        self.wk = nn.Linear(dim, dim, bias=qkv_bias)
        self.wv = nn.Linear(dim, dim, bias=qkv_bias)
        self.attn_drop = nn.Dropout(attn_drop)
        self.proj = nn.Linear(dim, dim)
        self.proj_drop = nn.Dropout(proj_drop)
        self.patch_size = patch_size

    def forward(self, x):
        B, N, C = x.shape
        
        # Compute Q, K, V matrices
        q = self.wq(x[:, 0:1, ...]).reshape(B, 1, self.num_heads, C // self.num_heads).permute(0, 2, 1, 3)
        k = self.wk(x).reshape(B, N, self.num_heads, C // self.num_heads).permute(0, 2, 1, 3)
        v = self.wv(x).reshape(B, N, self.num_heads, C // self.num_heads).permute(0, 2, 1, 3)
        
        # Compute attention scores
        attn = (q @ k.transpose(-2, -1)) * self.scale
        attn = attn.softmax(dim=-1)
        attn = self.attn_drop(attn)

        # Compute attention output
        x = (attn @ v).transpose(1, 2).reshape(B, 1, C)
        x = self.proj(x)
        x = self.proj_drop(x)
        
        return x


In [9]:
class CrossAttentionBlock(nn.Module):
    def __init__(self, dim, num_heads, mlp_ratio=4., qkv_bias=False, qk_scale=None, drop=0., attn_drop=0.,
                 drop_path=0., act_layer=nn.GELU, norm_layer=nn.LayerNorm, has_mlp=True, patch_size=16):
        super().__init__()
        self.norm1 = norm_layer(dim)
        self.attn = CrossAttention(dim, num_heads=num_heads, qkv_bias=qkv_bias, qk_scale=qk_scale,
                                   attn_drop=attn_drop, proj_drop=drop, patch_size=patch_size)
        self.drop_path = DropPath(drop_path) if drop_path > 0. else nn.Identity()
        self.has_mlp = has_mlp
        if self.has_mlp:
            self.norm2 = norm_layer(dim)
            mlp_hidden_dim = int(dim * mlp_ratio)
            self.mlp = Mlp(in_features=dim, hidden_features=mlp_hidden_dim, act_layer=act_layer, drop=drop)

    def forward(self, x):
        x = x[:, 0:1, ...] + self.drop_path(self.attn(self.norm1(x)))
        if self.has_mlp:
            x = x + self.drop_path(self.mlp(self.norm2(x)))
        return x


In [10]:
class MultiScaleBlock(nn.Module):
    def __init__(self, dim, patches, depth, num_heads, mlp_ratio, act_layer=nn.GELU, qkv_bias=False, 
                 qk_scale=None, attn_drop=0., drop=0., norm_layer=nn.LayerNorm):
        super().__init__()

        # Creating branches based on the embedding dimension
        num_branches = len(dim)
        self.num_branches = num_branches

        # Transformer block for each branch
        self.blocks = nn.ModuleList()
        for d in range(num_branches):
            temp = []
            for i in range(depth[d]):
                temp.append(
                    Block(dim=dim[d], num_heads=num_heads[d], mlp_ratio=mlp_ratio[d],
                          qkv_bias=qkv_bias, drop=drop, attn_drop=attn_drop, 
                          drop_path=drop_path[i], norm_layer=norm_layer)
                )
            if len(temp) != 0:
                self.blocks.append(nn.Sequential(*temp))
        if len(self.blocks) == 0:
            self.blocks = None

        # Ensuring that all branches are of the same size, creating projection layers if so
        self.proje = nn.ModuleList()
        for d in range(num_branches):
            temp = [norm_layer(dim[d]), act_layer(), nn.Linear(dim[d], dim[(d+1) % num_branches])]
            self.proje.append(nn.Sequential(*temp))

        self.fusion = nn.ModuleList()
        for d in range(num_branches):
            d_ = d + 1 % (num_branches)
            nh = num_heads[d_]
            if depth[-1] == 0:
                self.fusion.append(CrossAttentionBlock(dim=dim[d_], num_heads=nh, mlp_ratio=mlp_ratio[d],
                                                       qkv_bias=qkv_bias, qk_scale=qk_scale, drop=drop, 
                                                       attn_drop=attn_drop, drop_path=drop_path[-1], 
                                                       norm_layer=norm_layer, has_mlp=False, 
                                                       patch_size=patches[d_]))
            else:
                temp = []
                for _ in range(depth[-1]):
                    temp.append(CrossAttentionBlock(dim=dim[d_], num_heads=nh, mlp_ratio=mlp_ratio[d], 
                                                    qkv_bias=qkv_bias, qk_scale=qk_scale, drop=drop, 
                                                    attn_drop=attn_drop, drop_path=drop_path[-1], 
                                                    norm_layer=norm_layer, has_mlp=False, 
                                                    patch_size=patches[d_]))
                self.fusion.append(nn.Sequential(*temp))

        self.revert_projs = nn.ModuleList()
        for d in range(num_branches):
            temp = [norm_layer(dim[(d+1) % num_branches]), act_layer(), 
                    nn.Linear(dim[(d+1) % num_branches], dim[d])]
            self.revert_projs.append(nn.Sequential(*temp))

    def forward(self, x):
        outs_b = [blocks(x_) for x_, blocks in zip(x, self.blocks)]
        proj_cls_tok = [proj(x[:, 0:1]) for x, proj in zip(outs_b, self.proje)]

        outs = []
        for i in range(self.num_branches):
            temp = torch.cat((proj_cls_tok[i], outs_b[(i + 1) % self.num_branches][:, 1:, ...]), dim=1)
            temp = self.fusion[i](temp)
            reverted_pro_cls_tok = self.revert_projs[i](temp[:, 0:1, ...])
            temp = torch.cat((reverted_pro_cls_tok, outs_b[i][:, 0:1, ...]), dim=1)
        return outs


In [ ]:
import os
import torch
from torchvision import transforms, datasets
from torch.utils.data import DataLoader, Dataset
from PIL import Image

# Define the paths to your datasets
data_dir = '/Users/naren/vit/jetson-inference/python/training/classification/data/PlantCLEF_Subset'  # Directory containing 'train', 'val', and 'test' folders

# Extract class labels from the 'train' folder
train_dir = os.path.join(data_dir, 'train')
classes = sorted(entry.name for entry in os.scandir(train_dir) if entry.is_dir())

# Define a custom dataset class to handle the test set
class TestDataset(Dataset):
    def __init__(self, root, transform=None):
        self.root = root
        self.transform = transform
        self.image_paths = [os.path.join(root, fname) for fname in os.listdir(root)]
        
    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        return image, -1  # return a dummy label since the test set doesn't have labels

# Define transformations for training and validation
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

val_test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Load datasets
train_dataset = datasets.ImageFolder(root=train_dir, transform=train_transform)
val_dataset = datasets.ImageFolder(root=os.path.join(data_dir, 'val'), transform=val_test_transform)
test_dataset = TestDataset(root=os.path.join(data_dir, 'test'), transform=val_test_transform)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=4)

# Verify the class labels
print("Class labels:", train_dataset.classes)


Class labels: ['ash', 'beech', 'cattail', 'cedar', 'clover', 'cyprus', 'daisy', 'dandelion', 'dogwood', 'elm', 'fern', 'fig', 'fir', 'juniper', 'maple', 'poison_ivy', 'sweetgum', 'sycamore', 'trout_lily', 'tulip_tree']


In [12]:
import torch
import torch.nn as nn
import torch.optim as optim

# Define the Block class
class Block(nn.Module):
    def __init__(self, dim, num_heads, mlp_ratio=4., qkv_bias=False, qk_scale=None, drop=0., attn_drop=0., drop_path=0., act_layer=nn.GELU, norm_layer=nn.LayerNorm):
        super().__init__()
        self.norm1 = norm_layer(dim)
        self.attn = nn.MultiheadAttention(embed_dim=dim, num_heads=num_heads, dropout=attn_drop)
        self.drop_path = nn.Identity() if drop_path == 0 else nn.Dropout(drop_path)
        self.norm2 = norm_layer(dim)
        mlp_hidden_dim = int(dim * mlp_ratio)
        self.mlp = nn.Sequential(
            nn.Linear(dim, mlp_hidden_dim),
            act_layer(),
            nn.Dropout(drop),
            nn.Linear(mlp_hidden_dim, dim),
            nn.Dropout(drop),
        )

    def forward(self, x):
        x = x + self.drop_path(self.attn(self.norm1(x), self.norm1(x), self.norm1(x))[0])
        x = x + self.drop_path(self.mlp(self.norm2(x)))
        return x

# Define the MultiScaleBlock class
class MultiScaleBlock(nn.Module):
    def __init__(self, dim, patches, depth, num_heads, mlp_ratio, act_layer=nn.GELU, qkv_bias=False, qk_scale=None, attn_drop=0., drop=0., drop_path=None, norm_layer=nn.LayerNorm):
        super().__init__()
        self.blocks = nn.ModuleList()
        temp = []
        for d in range(len(dim)):
            for i in range(depth[d]):
                temp.append(
                    Block(
                        dim=dim[d],
                        num_heads=num_heads[d],
                        mlp_ratio=mlp_ratio[d],
                        qkv_bias=qkv_bias,
                        drop=drop,
                        attn_drop=attn_drop,
                        drop_path=drop_path[i],  # Ensure drop_path is indexed correctly
                        norm_layer=norm_layer
                    )
                )
            if len(temp) != 0:
                self.blocks.append(nn.Sequential(*temp))

    def forward(self, x):
        for blk in self.blocks:
            x = blk(x)
        return x

# Define the Vision Transformer model with classification head
class VisionTransformer(nn.Module):
    def __init__(self, img_size, patch_size, in_chans, num_classes, embed_dim, depth, num_heads, mlp_ratio, qkv_bias=True, qk_scale=None, drop_rate=0., attn_drop_rate=0., drop_path_rate=0., hybrid_backbone=None, norm_layer=nn.LayerNorm, multi_conv=False):
        super().__init__()
        self.num_classes = num_classes
        self.embed_dim = embed_dim

        # Flatten the depth list of lists
        flat_depth = [item for sublist in depth for item in sublist]

        # Calculate dpr based on the flattened depth list
        dpr = [x.item() for x in torch.linspace(0, drop_path_rate, sum(flat_depth))]

        self.blocks = nn.ModuleList([
            MultiScaleBlock(
                dim=embed_dim,
                patches=patch_size,
                depth=block_cfg,
                num_heads=num_heads,
                mlp_ratio=mlp_ratio,
                qkv_bias=qkv_bias,
                qk_scale=qk_scale,
                drop=drop_rate,
                attn_drop=attn_drop_rate,
                drop_path=dpr,  # Pass the drop path here
                norm_layer=norm_layer
            )
            for block_cfg in depth
        ])

        self.norm = nn.ModuleList([norm_layer(d) for d in embed_dim])
        self.head = nn.ModuleList([nn.Linear(d, num_classes) if num_classes > 0 else nn.Identity() for d in embed_dim])

        for i in range(len(embed_dim)):
            trunc_normal_(self.pos_embed[i], std=.02)
            trunc_normal_(self.cls_token[i], std=.02)

        self.apply(self._init_weights)

    def _init_weights(self, m):
        if isinstance(m, nn.Linear):
            trunc_normal_(m.weight, std=.02)
            nn.init.constant_(m.bias, 0)
        elif isinstance(m, nn.LayerNorm):
            nn.init.constant_(m.bias, 0)
            nn.init.constant_(m.weight, 1.0)

    def forward_features(self, x):
        B, C, H, W = x.shape
        xs = []
        for i in range(len(self.embed_dim)):
            x_ = torch.nn.functional.interpolate(x, size=(self.img_size[i], self.img_size[i]), mode='bicubic') if H != self.img_size[i] else x
            tmp = self.patch_embed[i](x_)
            cls_tokens = self.cls_token[i].expand(B, -1, -1)
            tmp = torch.cat((cls_tokens, tmp), dim=1)
            tmp = tmp + self.pos_embed[i]
            tmp = self.pos_drop(tmp)
            xs.append(tmp)

        for blk in self.blocks:
            xs = blk(xs)

        xs = [self.norm[i](x) for i, x in enumerate(xs)]
        out = [x[:, 0] for x in xs]

        return out

    def forward(self, x):
        xs = self.forward_features(x)
        ce_logits = [self.head[i](x) for i, x in enumerate(xs)]
        ce_logits = torch.mean(torch.stack(ce_logits, dim=0), dim=0)
        return ce_logits


In [14]:
# Function to train one epoch
def train_one_epoch(model, data_loader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in data_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    epoch_loss = running_loss / len(data_loader.dataset)
    epoch_accuracy = correct / total
    return epoch_loss, epoch_accuracy

# Function to validate the model
def validate(model, data_loader, criterion, device):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in data_loader:
            images, labels = images.to(device), labels.to(device)

            outputs = model(images)


In [16]:
import torch

img_size = 224
patch_sizes = [24, 12, 4]
in_chan = 3
embed_dim = 768
num_heads = 8

for patch_size in patch_sizes:
    print(f"\nTesting with patch size: {patch_size}")

    # Initialize the PatchEmbed layer
    patch_embed = PatchEmbed(img_size=img_size, patch_size=patch_size, in_chan=in_chan, embed_dim=embed_dim, multi_conv=True)

    # Generate a random input tensor simulating an image
    x = torch.randn(1, in_chan, img_size, img_size)
    print(f"Input shape: {x.shape}")

    # Generate patch embeddings and pad if necessary
    patch_embeddings = patch_embed(x, extra_padding=True)

    # Check the shape of the patch_embeddings tensor
    print(f"Patch embeddings raw shape: {patch_embeddings.shape}")

    if patch_embeddings.ndimension() == 4:  # If it has the shape (B, C, H', W')
        B, C, H_prime, W_prime = patch_embeddings.shape
        print(f"Padded shape: {x.shape}")
        print(f"Patch embeddings shape: {patch_embeddings.shape}")

        # Reshape for CrossAttention input
        N = H_prime * W_prime  # Number of patches
        patch_embeddings = patch_embeddings.permute(0, 2, 3, 1).reshape(B, N, C)  # (B, H', W', C) -> (B, N, C)
    elif patch_embeddings.ndimension() == 3:  # If it has already been flattened to (B, N, C)
        B, N, C = patch_embeddings.shape
        print(f"Padded shape: {x.shape} (no further reshaping required)")
        print(f"Patch embeddings shape: {patch_embeddings.shape}")

    # Initialize and apply the CrossAttention layer
    cross_attention = CrossAttention(dim=embed_dim, num_heads=num_heads, patch_size=patch_size)
    attention_output = cross_attention(patch_embeddings)
    print(f"Attention output shape: {attention_output.shape}")



Testing with patch size: 24
Input shape: torch.Size([1, 3, 224, 224])
Input shape: torch.Size([1, 3, 224, 224])
Padded shape: torch.Size([1, 3, 240, 240])
Patch embeddings raw shape: torch.Size([1, 1600, 768])
Padded shape: torch.Size([1, 3, 224, 224]) (no further reshaping required)
Patch embeddings shape: torch.Size([1, 1600, 768])
Attention output shape: torch.Size([1, 1, 768])

Testing with patch size: 12
Input shape: torch.Size([1, 3, 224, 224])
Input shape: torch.Size([1, 3, 224, 224])
Padded shape: torch.Size([1, 3, 228, 228])
Patch embeddings raw shape: torch.Size([1, 841, 768])
Padded shape: torch.Size([1, 3, 224, 224]) (no further reshaping required)
Patch embeddings shape: torch.Size([1, 841, 768])
Attention output shape: torch.Size([1, 1, 768])
